In [4]:
import baostock as bs
import pandas as pd
from datetime import datetime, timedelta, date

lg = bs.login()

login success!


In [5]:
#### 获取所有上市公司证券编码 ####
all_stock_code = []
all_stock_res = bs.query_stock_basic()
while (all_stock_res.error_code == '0') & all_stock_res.next():
  stock_data = all_stock_res.get_row_data()
  # 类型为股票且未退市的
  if (stock_data[4] == '1' and stock_data[5] == '1'):
    all_stock_code.append(stock_data[0])
print('所有在市股票数量为：', len(all_stock_code))

所有在市股票数量为： 4930


In [6]:
#### 取近五年所有的季度财报 ####
years = [2018, 2019, 2020, 2021, 2022]
quarters = [1, 2, 3, 4]
fields = [
  '编码',
  '名称',
  '年份',
  '季度',
  '财报发布日期',
  '持有时长(年)',
  '资产负债率',
  '现金流比率',
  '市净率',
  '市净率相对值',
  '发布时股价',
  '当前股价',
  '收益率',
  '年化收益率',
]

In [7]:
#### 获取财报公布的最近的交易日 ####
def getIsTradingDate(currDate):
  result = False
  rs = bs.query_trade_dates(start_date=currDate, end_date=currDate)
  while (rs.error_code == '0') & rs.next():
    rowData = rs.get_row_data()
    if (rowData[1] == '1'):
      result = True
      return result
  return result

In [8]:
#### 获取财报发布最近的交易日 ####
def getClosestDate(pubDate):
  isTradingDate = getIsTradingDate(pubDate)
  if (isTradingDate):
    return pubDate
  result = pubDate
  date_obj = datetime.strptime(pubDate, '%Y-%m-%d')
  prevDate = (date_obj + timedelta(days=-2)).strftime('%Y-%m-%d')
  nextDate = (date_obj + timedelta(days=+2)).strftime('%Y-%m-%d')
  rs = bs.query_trade_dates(start_date=prevDate, end_date=nextDate)
  while (rs.error_code == '0') & rs.next():
    rowData = rs.get_row_data()
    if (rowData[1] == '1'):
      result = rowData[0]
      return result
  return pubDate

In [9]:
#### 计算持有时长 ####
def getHoldingYears(currDate):
  date_obj = datetime.strptime(currDate, '%Y-%m-%d').date()
  # 计算日期到今天为止的年限
  today = date.today()
  delta = today - date_obj
  years = delta.days / 365.25
  return years

In [10]:
#### 获取当前日期 ####
def getToday():
  today = date.today().strftime('%Y-%m-%d')
  return getClosestDate(today)

In [11]:
#### 获取当前PB和历史最低PB的差距 ####
def getPBPercentToMin(code, date):
  try:
    raw_prev_date = datetime.strptime(date, '%Y-%m-%d') + timedelta(days=-1095)
    prev_three_years_date = raw_prev_date.strftime('%Y-%m-%d') if isinstance(raw_prev_date, datetime) else date
    rs = bs.query_history_k_data_plus(
      code,
      "pbMRQ",
      start_date=prev_three_years_date,
      end_date=date, 
      frequency="d",
      adjustflag="1",
    )
    data_array = []
    while (rs.error_code == '0') & rs.next():
      # 获取一条记录，将记录合并在一起
      data_array.append(rs.get_row_data())
    result = pd.DataFrame(data_array, columns=rs.fields)
    dropedResult = result.replace('', pd.np.nan).dropna(subset=['pbMRQ'])
    minPB = dropedResult['pbMRQ'].astype(float).min()
    maxPB = dropedResult['pbMRQ'].astype(float).max()
    currentPB = float(dropedResult['pbMRQ'][len(dropedResult) - 1])
    percentToMin = (currentPB - minPB) / (maxPB - minPB)
    return percentToMin
  except:
    return 10000

In [12]:
#### 获取财报数据 #### 
data_list = []
def getStockData(code, year, quarter):
  record = [code]
  closestDate = ''
  pub_close = 0
  current_close = 0
  holdingYears = 0
  # 证券名称
  stockInfoRes = bs.query_stock_basic(code)
  if (stockInfoRes.error_code == '0') & stockInfoRes.next():
    code_name = stockInfoRes.get_row_data()[1]
    record.extend([code_name, year, quarter])
  # 财报发布日期
  pubDateRes = bs.query_profit_data(code, year, quarter)
  if (pubDateRes.error_code == '0') & pubDateRes.next():
    pubDate = pubDateRes.get_row_data()[1]
    closestDate = getClosestDate(pubDate)
    holdingYears = getHoldingYears(closestDate)
    record.extend([closestDate, holdingYears])
  else:
    record.extend(['Empty', 'Empty'])
  # 资产负债率
  debtAssetRatioRes = bs.query_balance_data(code, year, quarter)
  if (debtAssetRatioRes.error_code == '0') & debtAssetRatioRes.next():
    debt_asset_ratio = debtAssetRatioRes.get_row_data()[7]
    record.append(float(debt_asset_ratio) if len(debt_asset_ratio) > 0 else 10000)
  else:
    record.append(-10000)
  # 现金流量比率
  cashFlowRatioRes = bs.query_cash_flow_data(code, year, quarter)
  if (cashFlowRatioRes.error_code == '0') & cashFlowRatioRes.next():
    cash_flow_ratio = cashFlowRatioRes.get_row_data()[7]
    record.append(float(cash_flow_ratio) if len(cash_flow_ratio) > 0 else -10000)
  else:
    record.append(-10000)
  # 市净率
  peTTMRes = bs.query_history_k_data_plus(
    code,
    "pbMRQ,close",
    start_date=closestDate,
    end_date=closestDate,
    frequency="d",
    adjustflag="1",
  )
  if (peTTMRes.error_code == '0') & peTTMRes.next():
    k_data = peTTMRes.get_row_data()
    pbMRQ = float(k_data[0]) if len(k_data[0]) > 0 else 10000
    pub_close = float(k_data[1])
    pb_to_min = getPBPercentToMin(code, closestDate) if len(k_data[0]) > 0 else 10000
    record.extend([pbMRQ, pb_to_min, pub_close])
  else :
    record.extend([10000, 10000, 0])
  # 当前股价
  currentRes = bs.query_history_k_data_plus(
    code,
    "close",
    start_date=getToday(),
    end_date=getToday(),
    frequency="d",
    adjustflag="1",
  )
  if (currentRes.error_code == '0') & currentRes.next():
    current_k_data = currentRes.get_row_data()
    current_close = float(current_k_data[0])
    record.extend([current_close])
  else :
    record.extend([0])
  # 收益率
  earning_ratio = (current_close - pub_close) / pub_close if (pub_close != 0) else 0
  # 年化收益率
  annual_earning_ratio = ((1 + earning_ratio) ** (1 / holdingYears) - 1) if (holdingYears != 0) else 0
  record.extend([earning_ratio, annual_earning_ratio])
  data_list.append(record)

In [13]:
#### 函数主流程 ####
def main():
  for index, code in enumerate(all_stock_code[:10]):
    print('当前进度', index + 1,'/ 4930，进度百分比为', (index + 1) / 4930 * 100, '%')
    for year in years:
      for quarter in quarters:
        getStockData(code, year, quarter)
  result = pd.DataFrame(data_list, columns=fields)
  print('result --->', result)

  result.to_csv("/Users/chenhao/Downloads/all_stock.csv", index=False)
  print('success !!')


In [14]:
main()

当前进度 1 / 500，进度百分比为 0.2 %
当前进度 2 / 500，进度百分比为 0.4 %
当前进度 3 / 500，进度百分比为 0.6 %
当前进度 4 / 500，进度百分比为 0.8 %
当前进度 5 / 500，进度百分比为 1.0 %
当前进度 6 / 500，进度百分比为 1.2 %
当前进度 7 / 500，进度百分比为 1.4000000000000001 %
当前进度 8 / 500，进度百分比为 1.6 %
当前进度 9 / 500，进度百分比为 1.7999999999999998 %
当前进度 10 / 500，进度百分比为 2.0 %
result --->             编码    名称    年份  季度      财报发布日期   持有时长(年)     资产负债率     现金流比率  \
0    sh.600000  浦发银行  2018   1  2018-04-26  5.086927  0.929103 -1.229529   
1    sh.600000  浦发银行  2018   2  2018-08-30  4.741958  0.926553 -1.435701   
2    sh.600000  浦发银行  2018   3  2018-10-31  4.572211  0.924122 -2.848571   
3    sh.600000  浦发银行  2018   4  2019-03-26  4.172485  0.923941 -1.972462   
4    sh.600000  浦发银行  2019   1  2019-04-30  4.076660  0.924334  3.702440   
..         ...   ...   ...  ..         ...       ...       ...       ...   
195  sh.600015  华夏银行  2021   4  2022-04-28  1.081451  0.918205  0.740743   
196  sh.600015  华夏银行  2022   1  2022-04-28  1.081451  0.920268  1.633292   
197  sh.60001